In [2]:
# Importacion de librerias
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 
#Tienes que descargarte las stopwords primero via nltk.download()
import nltk
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from nltk import word_tokenize
from nltk.data import load
from nltk.stem import SnowballStemmer 
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from string import punctuation
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict 
from sklearn import model_selection, naive_bayes, svm



In [15]:
# Importacion del archivo que se nos proporciono agregandole cabeceras que no contiene
names = ['Id', 'tweet', 'polaridad']
df = pd.read_csv('TweetsEtiquetadosPolaridad.txt', sep='\t',names=names)
df2 = pd.read_csv('TweetsEtiquetadosPolaridad.txt', sep='\t',names=names)

In [10]:
df.head(3)

,Id,tweet,polaridad
0,819337926026862593,Antojo de empanada colombiana,NEU
1,817464156756574208,Me volvieron a dejar sola,N
2,819237267181080576,@ManuTonic Buenotes diotas. Un gran abrazo,P


In [17]:

# Paso - 1c: Tokenización: en este cada entrada en el corpus se dividirá en un conjunto de palabras
df['tweet']= [word_tokenize(entry) for entry in df['tweet']]

# Paso - 1d: Eliminar palabras de detención, no numéricas y realizar el vástago / lemmenting de palabras.
# WordNetLemmatizer requiere etiquetas Pos para comprender si la palabra es sustantivo o verbo o adjetivo, etc. 
# De manera predeterminada, está configurada como Noun
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
for index,entry in enumerate(df['tweet']):
     # Declarar la lista vacía para almacenar las palabras que siguen las reglas de este paso
    Final_words = []
    # Inicializando WordNetLemmatizer ()
    word_Lemmatized = WordNetLemmatizer()
    # La función pos_tag a continuación proporcionará la 'etiqueta', es decir, si la palabra es Sustantivo (N) o Verbo (V) u otra cosa.
    for word, tag in pos_tag(entry):
        # La siguiente condición es verificar las palabras Stop y considerar solo alfabetos
        if word not in stopwords.words('spanish') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # El conjunto final procesado de palabras para cada iteración se almacenará en 'text_final'
    df.loc[index,'tweet_final'] = str(Final_words)
print(df.loc[index,'tweet_final'])

['compre', 'Hasta', 'Borinquen', 'internet', 'ver', 'si', 'aprende', 'poquito', 'dice', 'bobadas']


In [21]:
df.head(3)

,Id,tweet,polaridad,tweet_final
0,819337926026862593,"[Antojo, de, empanada, colombiana]",NEU,"['Antojo', 'empanada', 'colombiana']"
1,817464156756574208,"[Me, volvieron, a, dejar, sola]",N,"['Me', 'volvieron', 'dejar', 'sola']"
2,819237267181080576,"[@, ManuTonic, Buenotes, diotas, ., Un, gran, ...",P,"['ManuTonic', 'Buenotes', 'diotas', 'Un', 'gra..."


In [25]:
from nltk import ngrams 
n = 2
twograms = ngrams(df.loc[index,'tweet_final'], n)
for grams in twograms:
     print(grams)

('[', "'")
("'", 'A')
('A', 'n')
('n', 't')
('t', 'o')
('o', 'j')
('j', 'o')
('o', "'")
("'", ',')
(',', ' ')
(' ', "'")
("'", 'e')
('e', 'm')
('m', 'p')
('p', 'a')
('a', 'n')
('n', 'a')
('a', 'd')
('d', 'a')
('a', "'")
("'", ',')
(',', ' ')
(' ', "'")
("'", 'c')
('c', 'o')
('o', 'l')
('l', 'o')
('o', 'm')
('m', 'b')
('b', 'i')
('i', 'a')
('a', 'n')
('n', 'a')
('a', "'")
("'", ']')


In [24]:
from nltk import ngrams 
n = 3
threegrams = ngrams(df.loc[index,'tweet_final'], n)
for grams in threegrams:
     print(grams)

('[', "'", 'A')
("'", 'A', 'n')
('A', 'n', 't')
('n', 't', 'o')
('t', 'o', 'j')
('o', 'j', 'o')
('j', 'o', "'")
('o', "'", ',')
("'", ',', ' ')
(',', ' ', "'")
(' ', "'", 'e')
("'", 'e', 'm')
('e', 'm', 'p')
('m', 'p', 'a')
('p', 'a', 'n')
('a', 'n', 'a')
('n', 'a', 'd')
('a', 'd', 'a')
('d', 'a', "'")
('a', "'", ',')
("'", ',', ' ')
(',', ' ', "'")
(' ', "'", 'c')
("'", 'c', 'o')
('c', 'o', 'l')
('o', 'l', 'o')
('l', 'o', 'm')
('o', 'm', 'b')
('m', 'b', 'i')
('b', 'i', 'a')
('i', 'a', 'n')
('a', 'n', 'a')
('n', 'a', "'")
('a', "'", ']')


In [28]:
from nltk import ngrams 
n = 4
fourgrams = ngrams(df.loc[index,'tweet_final'], n)
for grams in fourgrams:
     print(grams)

('[', "'", 'A', 'n')
("'", 'A', 'n', 't')
('A', 'n', 't', 'o')
('n', 't', 'o', 'j')
('t', 'o', 'j', 'o')
('o', 'j', 'o', "'")
('j', 'o', "'", ',')
('o', "'", ',', ' ')
("'", ',', ' ', "'")
(',', ' ', "'", 'e')
(' ', "'", 'e', 'm')
("'", 'e', 'm', 'p')
('e', 'm', 'p', 'a')
('m', 'p', 'a', 'n')
('p', 'a', 'n', 'a')
('a', 'n', 'a', 'd')
('n', 'a', 'd', 'a')
('a', 'd', 'a', "'")
('d', 'a', "'", ',')
('a', "'", ',', ' ')
("'", ',', ' ', "'")
(',', ' ', "'", 'c')
(' ', "'", 'c', 'o')
("'", 'c', 'o', 'l')
('c', 'o', 'l', 'o')
('o', 'l', 'o', 'm')
('l', 'o', 'm', 'b')
('o', 'm', 'b', 'i')
('m', 'b', 'i', 'a')
('b', 'i', 'a', 'n')
('i', 'a', 'n', 'a')
('a', 'n', 'a', "'")
('n', 'a', "'", ']')


In [27]:
from nltk import ngrams 
n = 2
twograms = nltk.ngrams(df.loc[index,'tweet_final'], n)
for grams in twograms:
     print(grams)

('[', "'")
("'", 'A')
('A', 'n')
('n', 't')
('t', 'o')
('o', 'j')
('j', 'o')
('o', "'")
("'", ',')
(',', ' ')
(' ', "'")
("'", 'e')
('e', 'm')
('m', 'p')
('p', 'a')
('a', 'n')
('n', 'a')
('a', 'd')
('d', 'a')
('a', "'")
("'", ',')
(',', ' ')
(' ', "'")
("'", 'c')
('c', 'o')
('o', 'l')
('l', 'o')
('o', 'm')
('m', 'b')
('b', 'i')
('i', 'a')
('a', 'n')
('n', 'a')
('a', "'")
("'", ']')


In [36]:
text = df2.loc[index,'tweet']

In [38]:
from nltk import ngrams 
n = 4
nltk_tokens = nltk.word_tokenize(text) 
print("\nPor dos palabras==================")
print(list(nltk.ngrams(nltk_tokens,2)))
print("\nPor tres palabras==================")
print(list(nltk.ngrams(nltk_tokens,3)))
print("\nPor cuatro palabras==================")
print(list(nltk.ngrams(nltk_tokens,4)))
 


Por dos palabras==================
[('Antojo', 'de'), ('de', 'empanada'), ('empanada', 'colombiana')]

Por tres palabras==================
[('Antojo', 'de', 'empanada'), ('de', 'empanada', 'colombiana')]

Por cuatro palabras==================
[('Antojo', 'de', 'empanada', 'colombiana')]
